# Fast WOE

Author: https://github.com/deburky

In [1]:
import pandas as pd
from fastwoe import FastWoe
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
url = "https://raw.githubusercontent.com/georgianastasov/credit-bureau-2021-experian/refs/heads/main/score-model/BankCaseStudyData.csv"
df = pd.read_csv(url)

cat_features = [
    "Account_Type",
    "Promotion_Type",
    "Residential_Status",
    "Occupation_Code",
    "Age_of_Applicant",
    "Cheque_Card_Flag",
    "Existing_Customer_Flag",
    "Loan_Payment_Frequency",
    "Marital_Status",
    "Number_of_Payments",
    "SP_Number_Of_Searches_L6M",
]

X = df[cat_features].copy()
y = df['Final_Decision'].replace({"Accept": 0, "Decline": 1}).copy()

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/var/folders/k_/yz8rvp25185_js60dw8vhnj40000gn/T/ipykernel_26333/1812920356.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = df_ds['Final_Decision'].replace({"Accept": 0, "Decline": 1}).copy()


In [3]:
encoder = FastWoe(encoder_kwargs={"smooth": 1e-5, "cv": 2})
encoder.fit(X_train, y_train)

# Transform the data
X_train_woe = encoder.transform(X_train)
X_test_woe = encoder.transform(X_test)

# Make predictions
train_preds = pd.Series(encoder.predict_proba(X_train)[:, 1])
test_preds = pd.Series(encoder.predict_proba(X_test)[:, 1])

gini_train = roc_auc_score(y_train, train_preds) * 2 - 1
gini_test = roc_auc_score(y_test, test_preds) * 2 - 1
print(f"Gini Train: {gini_train:.4f}, Gini Test: {gini_test:.4f}")

# set pd set option to :.4f
pd.set_option('display.float_format', '{:.4f}'.format)
test_preds.describe()


Gini Train: 0.7531, Gini Test: 0.7434


count   4972.0000
mean       0.1587
std        0.2893
min        0.0000
25%        0.0009
50%        0.0078
75%        0.1360
max        0.9994
dtype: float64

In [4]:
encoder.get_all_mappings()['Account_Type']

,category,count,count_pct,event_rate,woe,good_count,bad_count
0,FL,13430,67.5316,0.1125,0.0959,11919,1510
1,VL,6457,32.4684,0.0841,-0.2267,5913,543


In [5]:
encoder.get_feature_stats()

,feature,n_categories,total_observations,missing_count,missing_rate,gini,information_value,min_woe,max_woe
0,Account_Type,2,19887,0,0.0000,0.0673,0.0217,-0.2267,0.0959
1,Promotion_Type,4,19887,0,0.0000,0.2058,0.2271,-1.9976,1.1195
2,Residential_Status,4,19887,0,0.0000,0.4392,0.7450,-0.9556,1.0738
3,Occupation_Code,4,19887,0,0.0000,0.2729,0.3011,-1.0470,0.7996
4,Age_of_Applicant,66,19887,0,0.0000,0.3420,0.4227,-14.3300,2.1613
5,Cheque_Card_Flag,3,19887,3,0.0002,0.4605,0.9392,-12.7206,0.8742
6,Existing_Customer_Flag,2,19887,0,0.0000,0.0607,0.0521,-0.7928,0.0660
7,Loan_Payment_Frequency,5,19887,1039,0.0522,0.3007,0.3425,-0.6975,0.5702
8,Marital_Status,5,19887,0,0.0000,0.2608,0.2653,-0.7861,0.5831
9,Number_of_Payments,42,19887,0,0.0000,0.4431,1.1526,-14.1068,2.7423
